# Analyzing River Thames Water Levels
Time series data is everywhere, from watching your stock portfolio to monitoring climate change, and even live-tracking as local cases of a virus become a global pandemic. In this project, you’ll work with a time series that tracks the tide levels of the Thames River. You’ll first load the data and inspect it data visually, and then perform calculations on the dataset to generate some summary statistics. You’ll end by decomposing the time series into its component attributes and analyzing them. 

The original dataset is available from the British Oceanographic Data Center [here](https://www.bodc.ac.uk/data/published_data_library/catalogue/10.5285/b66afb2c-cd53-7de9-e053-6c86abc0d251) and you can read all about this fascinating archival story in [this article](https://www.nature.com/articles/s41597-022-01223-7) from the Nature journal.

Here's a map of the locations of the tidal gauges along the River Thames in London.

![](locations.png)

The dataset comes with a file called `Data_description.pdf`. The dataset consists of 13 `.txt` files, containing comma separated data. We'll begin by analyzing one of them, the London Bridge gauge, and preparing it for analysis. The same code can be used to analyze data from other files (i.e. other gauges along the river) later.



| Variable Name | Description | Format |
| ------------- | ----------- | ------ |
| Date and time | Date and time of measurement to GMT. Note the tide gauge is accurate to one minute. | dd/mm/yyyy hh:mm:ss |
| Water level | High or low water level measured by tide gauge. Tide gauges are accurate to 1 centimetre. | metres (Admiralty Chart Datum (CD), Ordnance Datum Newlyn (ODN or Trinity High Water (THW)) | 
| Flag | High water flag = 1, low water flag = 0 | Categorical (0 or 1) |

In [1]:
# We've imported your first Python package for you, along with a function you will need called IQR
import pandas as pd               

def IQR(column): 
    q25, q75 = column.quantile([0.25, 0.75])
    return q75-q25

In [2]:
lb = pd.read_csv('data/10-11_London_Bridge.txt')

df = lb.iloc[:, :3]

df = df.rename(columns={'Date and time':'datetime', ' water level (m ODN)':'water_level', ' flag':'is_high_tide'})
df

,datetime,water_level,is_high_tide
0,01/05/1911 15:40:00,3.7130,1
1,02/05/1911 11:25:00,-2.9415,0
2,02/05/1911 16:05:00,3.3828,1
3,03/05/1911 11:50:00,-2.6367,0
4,03/05/1911 16:55:00,2.9256,1
...,...,...,...
115498,30/12/1995 20:44:00,3.2900,1
115499,31/12/1995 02:32:00,-1.6000,0
115500,31/12/1995 08:59:00,3.2000,1
115501,31/12/1995 15:03:00,-1.8000,0


In [3]:
df.dtypes

df['datetime'] = pd.to_datetime(df['datetime'])

df['water_level'] = df['water_level'].astype(float)

df['month'] = df['datetime'].dt.month

df['year'] = df['datetime'].dt.year

df

,datetime,water_level,is_high_tide,month,year
0,1911-01-05 15:40:00,3.7130,1,1,1911
1,1911-02-05 11:25:00,-2.9415,0,2,1911
2,1911-02-05 16:05:00,3.3828,1,2,1911
3,1911-03-05 11:50:00,-2.6367,0,3,1911
4,1911-03-05 16:55:00,2.9256,1,3,1911
...,...,...,...,...,...
115498,1995-12-30 20:44:00,3.2900,1,12,1995
115499,1995-12-31 02:32:00,-1.6000,0,12,1995
115500,1995-12-31 08:59:00,3.2000,1,12,1995
115501,1995-12-31 15:03:00,-1.8000,0,12,1995


In [4]:
tide_high = df.query("is_high_tide == 1")['water_level']

tide_low = df.query("is_high_tide == 0")['water_level']

high_statistics = tide_high.agg(['mean', 'median', IQR])

low_statistics = tide_low.agg(['mean', 'median', IQR])

In [5]:
all_high_days = df.query('is_high_tide==1').groupby('year').count()['water_level']

high_days = df.query(f'(water_level>{tide_high.quantile(.75)}) & (is_high_tide==1)').groupby('year').count()['water_level']

# Compute the ratio of high days to all high days
high_ratio = (high_days/all_high_days).reset_index()

In [6]:
all_low_days = df.query('is_high_tide==0').groupby('year').count()['water_level']

low_days = df.query(f'(water_level<{tide_low.quantile(.25)}) & (is_high_tide==0)').groupby('year').count()['water_level']

# Compute the ratio of high days to all high days
low_ratio = (low_days/all_low_days).reset_index()

In [7]:
solution = {'high_statistics': high_statistics, 'low_statistics': low_statistics, 'high_ratio': high_ratio, 'low_ratio':low_ratio}
print(solution)

{'high_statistics': mean      3.318373
median    3.352600
IQR       0.743600
Name: water_level, dtype: float64, 'low_statistics': mean     -2.383737
median   -2.412900
IQR       0.538200
Name: water_level, dtype: float64, 'high_ratio':     year  water_level
0   1911     0.032787
1   1912     0.127469
2   1913     0.186846
3   1914     0.161572
4   1915     0.219219
..   ...          ...
80  1991     0.252125
81  1992     0.265912
82  1993     0.317597
83  1994     0.357447
84  1995     0.324823

[85 rows x 2 columns], 'low_ratio':     year  water_level
0   1911     0.203463
1   1912     0.192793
2   1913     0.102985
3   1914     0.141618
4   1915     0.139818
..   ...          ...
80  1991     0.312057
81  1992     0.265912
82  1993     0.252496
83  1994     0.252482
84  1995     0.246809

[85 rows x 2 columns]}
